# Exploitation Zone - Model Predictiu

- exploitation zone del model predictiu
- preparation pipeline per taula d'entrenament del model --> cada zipcode és un indiv
    - Sales: 5 categories més comunes per zipcode, count vendes per zipcode, profit mitja per zipcode, mitjana num unitat per comanda per zipcode
    - Shops: 5shops més comunes per zipcode
    - Income: mitjana income per zipcode

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, mean, row_number
from pyspark.sql.window import Window
from pyspark.sql.functions import col, row_number, when, first
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder \
        .config("spark.jars", "./duckdb.jar") \
        .appName("ExploitationZone") \
        .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
24/04/25 03:49:30 WARN Utils: Your hostname, TABLET-F60ERQ04 resolves to a loopback address: 127.0.1.1; using 172.23.160.255 instead (on interface eth0)
24/04/25 03:49:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/25 03:49:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/25 03:49:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/25 03:49:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/04/25 03:49:34 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/04/25 03:49:34 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/04/25 03:49:34 WARN Utils: Service '

In [3]:
############
## INCOME ##
############

from pyspark.sql.functions import avg

average_income = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone.duckdb") \
    .option("dbtable", "cleaned_income") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .load() \
    .groupBy("ZIPCODE") \
    .agg(avg("Total_income_amount").alias("Average_Income"))

# Mostrar el resultado
average_income.show()

+-------+--------------+
|ZIPCODE|Average_Income|
+-------+--------------+
|  35079|   168362.0000|
|  35442|    69990.0000|
|  36269|    17803.0000|
|  99503|   493725.0000|
|  72044|    16750.0000|
|  72348|    32734.0000|
|  72533|     4427.0000|
|  90250|  1744745.0000|
|  90640|  1185590.0000|
|  91745|  1513989.0000|
|  91766|  1049937.0000|
|  92377|   478508.0000|
|  92591|  1081072.0000|
|  95005|   231483.0000|
|  81521|   390900.0000|
|   6811|  1123048.0000|
|  19803|  1192053.0000|
|  32096|    43647.0000|
|  32277|   586959.0000|
|  32542|    44570.0000|
+-------+--------------+
only showing top 20 rows



In [4]:
###########
## SALES ##
###########
from pyspark.sql.functions import col


sales_usa = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone.duckdb") \
    .option("dbtable", "cleaned_sales") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .load() \
    .filter(col("Country_/_Region") == "United States of America") \
    .drop("Customer_Name") \
    .select("ZIPCODE", "Category", "Sales", "Order_Quantity")

# Ventana para contar las categorías más frecuentes por código postal
category_window = Window.partitionBy("ZIPCODE").orderBy(col("count").desc())
df_category = sales_usa.groupBy("ZIPCODE", "Category").count()
df_category = df_category.withColumn("rn", row_number().over(category_window))

# Filtrar solo las top 5 categorías y pivotear
df_category = df_category.filter(col("rn") <= 5)
pivot_category = df_category.groupBy("ZIPCODE").pivot("rn", [1, 2, 3]).agg(first("Category"))

# Renombrar las columnas para que sean más descriptivas
b = pivot_category.select(
    col("ZIPCODE"),
    col("1").alias("Category_mas_comun_1"),
    col("2").alias("Category_mas_comun_2"),
    col("3").alias("Category_mas_comun_3")
)

df_averages = sales_usa.groupBy("ZIPCODE").agg(
    mean("Sales").alias("Average_Sales"),
    mean("Order_Quantity").alias("Average_Order_Quantity")
)


result_df = b.join(df_averages, "ZIPCODE", "outer")

result_df.show()


+-----------+--------------------+--------------------+--------------------+-------------+----------------------+
|Postal_Code|Category_mas_comun_1|Category_mas_comun_2|Category_mas_comun_3|Average_Sales|Average_Order_Quantity|
+-----------+--------------------+--------------------+--------------------+-------------+----------------------+
|     1001.0|Storage & Organiz...|                NULL|                NULL|        79.05|                5.0000|
|     1007.0|Computer Peripherals|                NULL|                NULL|         39.6|                2.0000|
|     1013.0|          Appliances|                NULL|                NULL|        717.6|               13.0000|
|     1027.0| Pens & Art Supplies|                NULL|                NULL|        35.28|               12.0000|
|     1028.0|Binders and Binde...|                NULL|                NULL|        57.96|                3.0000|
|     1040.0|Storage & Organiz...|                NULL|                NULL|       566.2

In [5]:
###########
## SHOP ##
###########

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, row_number, first
from pyspark.sql.window import Window

# Asumir que SparkSession ya está inicializado como 'spark'

# Cargar los datos y seleccionar columnas requeridas en un paso
shops = spark.read.format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone.duckdb") \
    .option("dbtable", "cleaned_shops") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .load() \
    .select("ZIPCODE", "shop")

# Calcular la cantidad de cada tipo de tienda por código postal y seleccionar las 5 principales
window_count = Window.partitionBy("ZIPCODE").orderBy(col("count").desc())
top_shops = shops.groupBy("ZIPCODE", "shop").count() \
    .withColumn("row_num", row_number().over(window_count)) \
    .filter(col("row_num") <= 5) \
    .drop("row_num") \
    .orderBy("ZIPCODE")

# Pivoteamos el DataFrame para tener cada tienda en una columna separada
window_pivot = Window.partitionBy("ZIPCODE").orderBy("count")
pivot_df = top_shops.withColumn("rn", row_number().over(window_pivot)) \
    .groupBy("ZIPCODE").pivot("rn", [1, 2, 3, 4, 5]).agg(first("shop"))

# Renombrar las columnas para hacerlas más descriptivas
pivot_df = pivot_df.select(
    col("ZIPCODE").alias("ZIPCODE"),
    col("1").alias("shop_mas_comun_1"),
    col("2").alias("shop_mas_comun_2"),
    col("3").alias("shop_mas_comun_3"),
    col("4").alias("shop_mas_comun_4"),
    col("5").alias("shop_mas_comun_5")
)

# Mostrar el resultado final
pivot_df.show()


+--------+----------------+----------------+-----------------+----------------+----------------+
|postcode|shop_mas_comun_1|shop_mas_comun_2| shop_mas_comun_3|shop_mas_comun_4|shop_mas_comun_5|
+--------+----------------+----------------+-----------------+----------------+----------------+
|   28000|         florist|        copyshop|      supermarket|        boutique|    dry_cleaning|
|   28017|           paint|     hairdresser|           beauty|     convenience|         clothes|
|   28020|      car_repair|            NULL|             NULL|            NULL|            NULL|
|   28035|           shoes|      stationery|          laundry|           paint|     convenience|
|   28040|department_store|            NULL|             NULL|            NULL|            NULL|
|   28450|          coffee|            NULL|             NULL|            NULL|            NULL|
|   28459|           shoes|         butcher|      hairdresser|             yes|     convenience|
|   28460|          bakery|   